### Import Modules & Basic Training

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

In [10]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

print("train_images.shape(before): ", train_images.shape)
print("test_images.shape(before): ", test_images.shape)

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

print("train_images.shape(after): ", train_images.shape)
print("test_images.shape(after): ", test_images.shape)

train_images.shape(before):  (60000, 28, 28)
test_images.shape(before):  (10000, 28, 28)
train_images.shape(after):  (1000, 784)
test_images.shape(after):  (1000, 784)


In [14]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## ModelCheckPoint Callback

In [16]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, train_labels,
          epochs=10, validation_data=(test_images, test_labels),
          callbacks=[cp_callback])   # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 1s 12ms/step - loss: 1.1597 - accuracy: 0.6620 - val_loss: 0.7416 - val_accuracy: 0.7650

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 0.4282 - accuracy: 0.8740 - val_loss: 0.5763 - val_accuracy: 0.8190

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 0.2960 - accuracy: 0.9370 - val_loss: 0.4665 - val_accuracy: 0.8530

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2305 - accuracy: 0.9420 - val_loss: 0.4581 - val_accuracy: 0.8560

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 0.1676 - accuracy: 0.9690 - val_loss: 0.4257 - val_accuracy: 0.8660

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s

In [18]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


### Create New Model

In [21]:
model = create_model()

loss, acc= model.evaluate(test_images, test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도 :{:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4224 - accuracy: 0.0770
훈련되지 않은 모델의 정확도 : 7.70%


### Reload Weights from Check Point

In [22]:
# 가중치 로드
model.load_weights(checkpoint_path)
# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 :{:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4142 - accuracy: 0.8650
복원된 모델의 정확도 :86.50%


## Parameter of Checkpoint

### 5번째 epoch마다 checkpoint를 저장하는 callback 정의

In [26]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}}.ckpt"   # 04d : 4칸으로 맞추고, 부족하면 앞에 0 붙임
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

### Training & Save Checkpoints

In [27]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))  # 모델의 초기 weight 저장

model.fit(train_images, train_labels,
          epochs=50, callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [30]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [31]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

### checkpoint load

In [34]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5031 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%


## 수동으로 가중치 저장

In [35]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5031 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%


## 전체 모델 저장하기

케라스는 모델 구조를 파악하여 저장합니다. 이 방식은 다음과 같은 모든 것을 저장합니다:

가중치 값  
모델 구조  
모델의 훈련 설정(컴파일할 때 전달한 값)  
옵티마이저와 필요하면 옵티마이저 상태(중지된 지점부터 다시 시작할 수 있습니다)

### SavedModel format

In [45]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 11ms/step - loss: 1.1710 - accuracy: 0.6680 - val_loss: 0.7153 - val_accuracy: 0.7840
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4373 - accuracy: 0.8820 - val_loss: 0.5266 - val_accuracy: 0.8470
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2943 - accuracy: 0.9180 - val_loss: 0.4922 - val_accuracy: 0.8380
Epoch 4/5
32/32 [==============================] - 0s 9ms/step - loss: 0.2189 - accuracy: 0.9500 - val_loss: 0.4503 - val_accuracy: 0.8560
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1530 - accuracy: 0.9640 - val_loss: 0.4290 - val_accuracy: 0.8610
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [46]:
!ls saved_model
!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [47]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [48]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 : {:5.2f}%".format(100*acc))

print("predicted value shape :", new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4290 - accuracy: 0.8610
복원된 모델의 정확도 : 86.10%
predicted value shape : (1000, 10)


### HDF5 format

In [49]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels,
          validation_data = (test_images, test_labels),
          epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 20ms/step - loss: 1.1951 - accuracy: 0.6630 - val_loss: 0.7252 - val_accuracy: 0.7900
Epoch 2/5
32/32 [==============================] - 0s 12ms/step - loss: 0.4255 - accuracy: 0.8860 - val_loss: 0.5415 - val_accuracy: 0.8400
Epoch 3/5
32/32 [==============================] - 0s 12ms/step - loss: 0.2878 - accuracy: 0.9310 - val_loss: 0.4987 - val_accuracy: 0.8370
Epoch 4/5
32/32 [==============================] - 0s 12ms/step - loss: 0.2103 - accuracy: 0.9470 - val_loss: 0.4382 - val_accuracy: 0.8600
Epoch 5/5
32/32 [==============================] - 0s 13ms/step - loss: 0.1589 - accuracy: 0.9690 - val_loss: 0.4132 - val_accuracy: 0.8670


In [50]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model("my_model.h5")

new_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [51]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 : {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4132 - accuracy: 0.8670
복원된 모델의 정확도 : 86.70%
